In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session|

ref: https://www.kaggle.com/code/dmitryfomkin/ts-lr-xgb-time-features-ny-pay-days

### Steps from the ref
1. train data and parse date + infer_date_format
    * use date.dt.to_period('D') -> Try not using this and see what's difference
2. Extract daily average sales and average promotion for later use.
3. Format the same for the testing data.
4. oil data parse_date =['date'] + infer_date_format
    * use date.dt.to_period('D') -> Try not using this and see what's difference
5. Holiday data parse_date =['date'] + infer_date_format
    * use date.dt.to_period('D') -> Try not using this and see what's difference
6. transaction data parse_date =['date'] + infer_date_format
    * use date.dt.to_period('D') -> Try not using this and see what's difference
7. Construct the LR model
    * X Feature
         * 0   weekofyear       227 non-null    int64  
         * 1   quarter          227 non-null    int64  
         * 2   month            227 non-null    int64  
         * 3   dayofyear        227 non-null    int64  
         * 4   dayofweek_0      227 non-null    uint8  
         * 5   dayofweek_1      227 non-null    uint8  
         * 6   dayofweek_2      227 non-null    uint8  
         * 7   dayofweek_3      227 non-null    uint8  
         * 8   dayofweek_4      227 non-null    uint8  
         * 9   dayofweek_5      227 non-null    uint8  
         * 10  dayofweek_6      227 non-null    uint8  
         * 11  new_year         227 non-null    int64  
         * 12  pay_day          227 non-null    bool   
         * 13  hol              227 non-null    float64
         * 14  type_Additional  227 non-null    float64
         * 15  type_Event       227 non-null    float64
         * 16  type_Holiday     227 non-null    float64
         * 17  type_Transfer    227 non-null    float64
         * 18  oil              227 non-null    float64
* y_ for training
    * Is an unstacked df of date index and ['store', 'family'] columns from multiindex and unstack store and family up.
    
8. Train model by writing a function to train the model
    * using cross_validate(scoring=['neg_mean_absolute_error','neg_mean_squared_log_error'],return_estimator=True)) and time_series_split
    * Try not to use the time_series_split. Just one run and see the result.
    
     


### Plots to draw
1. average sales between years in the same axis
2. Seasonal pattern in the same axis using loc year and groupby('dow')
3. dateplot for different holiday type using loc[df.column_name == 'str']
    * plot average sales in silver in the background too.
4. scaled transactions, oil price, oil price change, average onpromotion with average sales.

5. average sales fit and plot the prediction and validation of 2017-08-01 to 2017-08-15 by using cross_validate()


let's begin by

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt

In [ ]:
train_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv',parse_dates=['date'],infer_datetime_format=True)
#train_data['date'] = train_data.date.dt.to_period('D')

test_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv',parse_dates=['date'],infer_datetime_format=True)

oil_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv',parse_dates=['date'],infer_datetime_format=True)

holiday_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv',parse_dates=['date'],infer_datetime_format=True)
#holiday_data['date'] = holiday_data.date.dt.to_period('D')

transaction_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv',parse_dates=['date'],infer_datetime_format=True,index_col='date')
#transaction_data['date'] = transaction_data.date.dt.to_period('D')


In [ ]:
store_sales = train_data.set_index(['store_nbr','family','date'])
store_sales.tail()

In [ ]:
avg_sales = store_sales.groupby('date')['sales'].mean().to_frame()

In [ ]:
avg_prom = store_sales.groupby('date')['onpromotion'].mean().to_frame()

In [ ]:
oil_data.set_index('date', inplace=True)
oil_data = oil_data.resample('D').mean().fillna(method='ffill')

In [ ]:
oil_data

In [ ]:
transaction_data

### Plot to do
1. average sales between years in the same axis


In [ ]:
plt.figure(figsize=(18,5))
plt.plot(avg_sales.loc['2016'].values, label='2016')
plt.plot(avg_sales.loc['2017'].values, label ='2017')
plt.legend()
plt.show()

Seasonal or cycle patterns can be seen.

### Plot to do
2. Seasonal pattern in the same axis using loc year and groupby('dow')


In [ ]:
avg_sales['dow'] = avg_sales.index.weekday.to_list()

In [ ]:
fig = plt.figure()
for i in ['2013','2014','2015','2016','2017']:
    plt.plot(avg_sales.loc[i].groupby('dow').mean(), label=i)
plt.legend()
plt.show()

Sales are good in the weekend

### Plot to do

3. dateplot for different holiday type using loc[df.column_name == 'str']
    * plot average sales in silver in the background too.

In [ ]:
holiday_data.set_index('date',inplace=True)

In [ ]:
holiday_data

In [ ]:
#slize only holiday in the training data onwards

hol = holiday_data.loc[holiday_data.index.isin(avg_sales.index)]
hol_nat = hol.loc[(hol['locale']=='National') & 
                  (hol['transferred']!=True) & 
                  (hol['type']!='Work Day')]
hol_local = hol.loc[(hol['locale']=='Local') & 
                  (hol['transferred']!=True) & 
                  (hol['type']!='Work Day')]
hol_reg = hol.loc[(hol['locale']=='Regional') & 
                  (hol['transferred']!=True) & 
                  (hol['type']!='Work Day')]

fig = plt.figure(figsize=(8,5))
#plt.plot_date(hol.index,avg_sales.sales.loc[hol.index],color='red')
plt.plot_date(hol_nat.index,avg_sales.sales.loc[(hol_nat.index)],color='yellow',label='National')
plt.plot_date(hol_local.index,avg_sales.sales.loc[(hol_local.index)],color='green',label = 'Local')
plt.plot_date(hol_reg.index,avg_sales.sales.loc[(hol_reg.index)],color='blue',label = 'Regional')
plt.legend()
plt.plot(avg_sales['sales'],alpha=0.5,color='silver')
plt.show()

In [ ]:
test_data.set_index('date',inplace=True)


In [ ]:
#slice only holiday in the test data onwards

hol_test = holiday_data.loc[holiday_data.index.isin(test_data.index)]
hol_test

National (Yellow) seem to be more coincide with the peak of average daily sales.

### Plot to do
4. scaled transactions, oil price, oil price change, average onpromotion with average sales.


In [ ]:
from sklearn.preprocessing import MinMaxScaler

def mm_scale_column(df,colname):
    mm_scaler = MinMaxScaler()
    mm_scaler.fit(df[colname].values.reshape(-1,1))
    df['{}_scaled'.format(colname)] = mm_scaler.transform(df[colname].values.reshape(-1,1))
    return df

In [ ]:
oil_data.rename({'dcoilwtico':'oil_price'},axis=1,inplace=True)

In [ ]:
oil_data['price_chg'] = oil_data['oil_price']-oil_data['oil_price'].shift(1)

In [ ]:
oil_data.dropna(inplace=True)

In [ ]:
mm_scale_column(transaction_data,'transactions')
mm_scale_column(oil_data,'oil_price')
mm_scale_column(oil_data,'price_chg')
mm_scale_column(avg_prom,'onpromotion')
mm_scale_column(avg_sales,'sales')

In [ ]:
corr = transaction_data['transactions_scaled'].to_frame().corrwith(avg_sales['sales_scaled'])
print('corretaion = {}'.format(corr))

fig = plt.figure(figsize=(18,9))
plt.title('scaled average sales vs scaled transaction')
plt.plot(transaction_data['transactions_scaled'],label='scaled transaction')
plt.plot(avg_sales['sales_scaled'],label='scaled sales')
plt.legend()
plt.show()

In [ ]:
corr = avg_prom['onpromotion_scaled'].to_frame().corrwith(avg_sales['sales_scaled'])
print('corretaion = {}'.format(corr))

fig = plt.figure(figsize=(18,9))
plt.title('scaled average sales vs scaled promotion')
plt.plot(avg_prom['onpromotion_scaled'],label='scaled onpromotion')
plt.plot(avg_sales['sales_scaled'],label='scaled sales')
plt.legend()
plt.show()

In [ ]:
#oil_data.set_index('date',inplace=True)

In [ ]:
corr = oil_data['oil_price_scaled'].to_frame().corrwith(avg_sales['sales_scaled'])
print('corretaion = {}'.format(corr))

fig = plt.figure(figsize=(18,9))
plt.title('scaled average sales vs scaled oil price')
plt.plot(oil_data['oil_price_scaled'],label='oil_price_scaled')
plt.plot(avg_sales['sales_scaled'],label='scaled sales')
plt.legend()
plt.show()

In [ ]:
corr = oil_data['price_chg_scaled'].to_frame().corrwith(avg_sales['sales_scaled'])
print('corretaion = {}'.format(corr))

fig = plt.figure(figsize=(18,9))
plt.title('scaled average sales vs scaled oil price change')
plt.plot(oil_data['price_chg_scaled'],label='oil_price_change_scaled')
plt.plot(avg_sales['sales_scaled'],label='scaled sales')
plt.legend()
plt.show()

ref: https://www.kaggle.com/code/dmitryfomkin/ts-lr-xgb-time-features-ny-pay-days

### Let's recreate the ref LR model
1. train data and parse date + infer_date_format
    * use date.dt.to_period('D') -> Try not using this and see what's difference
2. Extract daily average sales and average promotion for later use.
3. Format the same for the testing data.
4. oil data parse_date =['date'] + infer_date_format
    * use date.dt.to_period('D') -> Try not using this and see what's difference
5. Holiday data parse_date =['date'] + infer_date_format
    * use date.dt.to_period('D') -> Try not using this and see what's difference
6. transaction data parse_date =['date'] + infer_date_format
    * use date.dt.to_period('D') -> Try not using this and see what's difference
7. Construct the LR model
    * X Feature
         * 0   weekofyear       227 non-null    int64  
         * 1   quarter          227 non-null    int64  
         * 2   month            227 non-null    int64  
         * 3   dayofyear        227 non-null    int64  
         * 4   dayofweek_0      227 non-null    uint8  
         * 5   dayofweek_1      227 non-null    uint8  
         * 6   dayofweek_2      227 non-null    uint8  
         * 7   dayofweek_3      227 non-null    uint8  
         * 8   dayofweek_4      227 non-null    uint8  
         * 9   dayofweek_5      227 non-null    uint8  
         * 10  dayofweek_6      227 non-null    uint8  
         * 11  new_year         227 non-null    int64  
         * 12  pay_day          227 non-null    bool   
         * 13  hol              227 non-null    float64
         * 14  type_Additional  227 non-null    float64
         * 15  type_Event       227 non-null    float64
         * 16  type_Holiday     227 non-null    float64
         * 17  type_Transfer    227 non-null    float64
         * 18  oil              227 non-null    float64
* y_ for training
    * Is an unstacked df of date index and ['store', 'family'] columns from multiindex and unstack store and family up.
    
8. Train model by writing a function to train the model
    * using cross_validate(scoring=['neg_mean_absolute_error','neg_mean_squared_log_error'],return_estimator=True)) and time_series_split
    * Try not to use the time_series_split. Just one run and see the result.

In [ ]:
# #time features
# X = avg_sales.copy()
# X.drop('sales_scaled',axis=1,inplace=True)
# X['weekofyear'] = X.index.isocalendar().week
# X['quarter'] = X.index.quarter
# X['month'] = X.index.month
# X['dayofyear'] = X.index.dayofyear
# X = pd.get_dummies(X, columns=['dow'],drop_first=True) #trying to not use drop_first

# #new year day that cause the sale to drop significantly
# X['is_ny'] = 0
# X.loc[X['dayofyear']==1, 'is_ny'] = 1

# #payday over bi-weekly
# X['payday'] = False
# X.loc[(X['month']!=X['month'].shift(-1)), 'payday'] = True
# X.loc[X.index.day == 15, 'payday'] = True

In [ ]:
#time features
X = avg_sales.copy()
X.drop('sales_scaled',axis=1,inplace=True)
X['weekofyear'] = X.index.isocalendar().week
X['quarter'] = X.index.quarter
X['month'] = X.index.month
X['dayofyear'] = X.index.dayofyear
X = pd.get_dummies(X, columns=['dow'])

#new year day that cause the sale to drop significantly
X['is_ny'] = 0
X.loc[X['dayofyear']==1, 'is_ny'] = 1

#payday over bi-weekly
X['payday'] = False
# X.loc[(X['month']!=X['month'].shift(-1)), 'payday'] = True
# X.loc[X.index.day == 15, 'payday'] = True
X['day_in_m'] = X.index.days_in_month
X['payday'] = (X.index.day == 16) | (X.index.day == 1) | (X.index.day == 14) | (X.index.day == X['day_in_m'] - 1) | (X.index.day == 15) | (X.index.day == X['day_in_m'])
X.drop(columns='day_in_m', inplace=True, axis=1)


In [ ]:
X.loc[X['payday'] == True]

Let's do the same formatting for test data.

In [ ]:
#time features
f_test = test_data.copy()
f_test['dow'] = f_test.index.weekday
f_test['weekofyear'] = f_test.index.isocalendar().week
f_test['quarter'] = f_test.index.quarter
f_test['month'] = f_test.index.month
f_test['dayofyear'] = f_test.index.dayofyear
f_test = pd.get_dummies(f_test, columns=['dow'])

#new year day that cause the sale to drop significantly
f_test['is_ny'] = 0
f_test.loc[f_test['dayofyear']==1, 'is_ny'] = 1

#payday over bi-weekly
f_test['payday'] = False
f_test.loc[(f_test['month']!=f_test['month'].shift(-1)), 'payday'] = True
f_test.loc[f_test.index.day == 31, 'payday'] = True

In [ ]:
#(pay_day_test.index.day == 16) | (pay_day_test.index.day == 1) | (pay_day_test.index.day == 14) | (pay_day_test.index.day == pay_day_test['day_in_m'] - 1) | (pay_day_test.index.day == 15) | (pay_day_test.index.day == pay_day_test['day_in_m'])

In [ ]:
f_test

In [ ]:
f_test.loc[f_test['payday'] == True]

To prepare holiday features need to work with the holiday_data first before concat into the X

In [ ]:
hol_X = hol.copy()
hol_X = hol.loc[hol.locale.isin(['National','Regional'])]

#do the same for test
hol_test = hol_test.loc[hol_test.locale.isin(['National','Regional'])]

In [ ]:
hol_X

In [ ]:
hol_test #no national or regional holiday during the test period

In [ ]:
hol_X['is_hol'] = 1
hol_X.loc[hol_X['transferred']==True, 'is_hol'] = 0
hol_X = pd.get_dummies(hol_X.drop(['locale','locale_name','description','transferred'], axis=1), columns=['type'])


In [ ]:
hol_X

Next, concat hol_X to X and f_test. Don't remember to set Sat and Sun to is_hol as well.

In [ ]:
X = X.merge(hol_X,how='left',left_index=True,right_index=True)

In [ ]:
X = X.fillna(0)
X.loc[X['type_Work Day'] == 1, 'is_hol'] = 0
X.loc[X['dow_5']==1, 'is_hol'] = 1
X.loc[X['dow_6']==1, 'is_hol'] = 1
X.drop('type_Work Day',axis=1,inplace=True)

In [ ]:
X

Let's do the same formatting for test data.

In [ ]:
f_test[['is_hol','type_Additional','type_Bridge','type_Event','type_Holiday','type_Transfer']] = 0
f_test.loc[f_test['dow_5']==1, 'is_hol'] = 1
f_test.loc[f_test['dow_6']==1, 'is_hol'] = 1

In [ ]:
f_test

Add oil data. Here I will comply whit the notebook I'm trying to recreate by using rolling(7).mean() of oil price

In [ ]:
oil_data['oil_price_ma7'] = oil_data['oil_price'].rolling(7).mean()

In [ ]:
X = X.merge(oil_data['oil_price_ma7'],how='left',left_index=True,right_index=True)

In [ ]:
f_test = f_test.merge(oil_data['oil_price_ma7'],how='left',left_index=True,right_index=True)

In [ ]:
X = X.dropna()
f_test = f_test.dropna()

In [ ]:
X

### Prepare y_ for training
y_ for training is an unstacked df of date index and ['store', 'family'] columns from multiindex and unstack store and family up.
    

In [ ]:
store_sales

In [ ]:
y_ = store_sales.dropna()['sales'].to_frame().unstack(['store_nbr','family'])
y_ = y_.loc['2013-01-09':]

In [ ]:
y_.loc['2017']

# The model

8. Train model by writing a function to train the model
    * using cross_validate(scoring=['neg_mean_absolute_error','neg_mean_squared_log_error'],return_estimator=True)) and time_series_split
    * Try not to use the time_series_split. Just one run and see the result.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
X_train = X.dropna().drop('sales',axis=1)
X_train = X_train[~X_train.index.duplicated()]

In [ ]:
X_train.tail()

In [ ]:
lr.fit(X_train, y_)

In [ ]:
f_test = f_test[~f_test.index.duplicated()]
f_test

In [ ]:
y_pred = pd.DataFrame(lr.predict(f_test.drop(['id','store_nbr','family','onpromotion'],axis=1)))
y_pred

In [ ]:
submit_df = pd.DataFrame(y_pred)
submit_df.columns = y_.columns
submit_df.index = f_test.index

In [ ]:
submit_df.tail()

In [ ]:
test_data.tail()

In [ ]:
submit_df = submit_df.stack(['store_nbr','family']).reset_index()
submit_df['id'] = np.arange(3000888,3029400)

In [ ]:
submit_df.loc[submit_df['sales'] < 0 ,'sales'] = 0
submit_df.head()

### Submission

In [ ]:
submit_file = submit_df[['id','sales']]
submit_file

In [ ]:
submit_df.loc[submit_df['id']==0]

In [ ]:
# submit_file.to_csv('submission.csv',index=False)

### Note on first submission

Submitted and got 1.14639

I think I could adjust something begining with one thing that's alway bother me, the non dummy weekofyear, quarter, month, dayofyear.

In [ ]:
X_train.corr().style.background_gradient('inferno')


They hava big correlation here. Let's drop all but day of year. and format it to have a category dtype.

In [ ]:
X_train['dayofyear'] = X_train['dayofyear'].astype('category')
X_train_doy = X_train.drop(['weekofyear','quarter','month'], axis=1)
f_test_drop = f_test.drop(['id','store_nbr','family','onpromotion','weekofyear','quarter','month'],axis=1)

In [ ]:
def run_model_set_zero(model,X_train,y_train,X_test,id_list):
    model.fit(X_train,y_train)
    y_pred = pd.DataFrame(model.predict(X_test))
    y_pred.index = X_test.index
    y_pred.columns = y_train.columns
    y_pred = y_pred.stack(['store_nbr','family'])
    y_pred.loc[y_pred['sales'] < 0, 'sales'] = 0
    y_pred['id'] = id_list
    return y_pred

In [ ]:
doy_result = run_model_set_zero(LinearRegression(),X_train_doy,y_,f_test_drop,test_data['id'].values)

In [ ]:
doy_result.head()

In [ ]:
# doy_result[['id','sales']].to_csv('submission.csv',index=False)

Submitted and got 1.14690 -> the score worsen a bit after cut time features.

### Next adjustment
Let's try scale the oil price ma and dayofyear just to see the result.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
def scale_column(df,column_list):
    x = df.copy()
    for col in column_list:
        s_scaler = StandardScaler()
        x[col] = s_scaler.fit_transform(x[col].to_frame())
    return x

In [ ]:
scaled_x_train_doy = scale_column(X_train_doy,['dayofyear','oil_price_ma7'])

In [ ]:
scaled_x_train_doy.tail()

In [ ]:
scaled_x_train_doy.describe()[['dayofyear','oil_price_ma7']]

In [ ]:
scaled_result = run_model_set_zero(LinearRegression(),scaled_x_train_doy,y_,f_test_drop,test_data['id'].values)
scaled_result

Let's see how the prediction looks visually.

In [ ]:
scaled_result.loc[:,3,'BEVERAGES']['sales'].size

In [ ]:
train_data.loc[(train_data['family'] =='BEVERAGES') & (train_data['store_nbr'] == 3)].sales.values

In [ ]:
extended_3_bev = np.hstack([train_data.loc[(train_data['family'] =='BEVERAGES') & (train_data['store_nbr'] == 3)].sales.values, scaled_result.loc[:,3,'BEVERAGES']['sales'].values])
extended_3_bev.size

In [ ]:
store_3_bev = train_data.loc[(train_data['family'] =='BEVERAGES') & (train_data['store_nbr'] == 3)]

fig = plt.figure()
# plt.plot(extended_3_bev)
plt.plot(extended_3_bev[:1686])
plt.plot(extended_3_bev[1686:])

let's compare the prediction from the first submission to this one.

In [ ]:
old_shop3_bev_pred = submit_df.set_index(['date','store_nbr','family']).loc[:,3,'BEVERAGES']['sales'].values

fig = plt.figure()
plt.plot(extended_3_bev[1686:],label='scaled_feature')
plt.plot(old_shop3_bev_pred,label='old_submission')
plt.legend()
plt.show()

The prediction on store 3 beverages sales differ from each other. Let's submit this version to see the test score.

In [ ]:
scaled_result[['id','sales']]

In [ ]:
#scaled_result[['id','sales']].to_csv('submission.csv',index=False)

### Somehow the score is so bad at 3.64565


### Next adjustment
Time feature seems to be valid features. Therefore, bring them back (without scale).
Also, Let's try cross_vaidate to evaluate with some metrics

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit

TS = TimeSeriesSplit()
model = LinearRegression()

In [ ]:
X_train.tail()

In [ ]:
result = cross_validate(estimator=model,
                        X=X_train,
                        y=y_,
                        cv=TS,
                        return_estimator=True,
                        #scoring=["neg_mean_absolute_error", "neg_mean_squared_log_error"]
                       )

In [ ]:
result

There are 5 instances of estimator returned from the cross validation. Let's try to use them to predict the outcome and compare them together.

In [ ]:
f_test.drop(['id','store_nbr','family','onpromotion'], axis=1)

In [ ]:
pred_dic = {}
for i,model in enumerate(result['estimator']):
    pred_dic['instance_{}_prediction'.format(i)] = model.predict(f_test.drop(['id','store_nbr','family','onpromotion'], axis=1))

In [ ]:
for key, array in pred_dic.items():
    pred_dic[key] = pd.DataFrame(array)
    pred_dic[key].columns = y_.columns
    pred_dic[key].index = f_test.index
    pred_dic[key] = pred_dic[key].stack(['store_nbr','family'])
    pred_dic[key].loc[pred_dic[key]['sales'] < 0, 'sales'] = 0

In [ ]:
pred_dic['instance_0_prediction']

Let's slice the shop 3 BEVERAGES sales to compare.

In [ ]:
shop3_bev_cv = {}

for key, df in pred_dic.items():
    shop3_bev_cv[key] = df.loc[:,3,'BEVERAGES']['sales'].values

In [ ]:
submit_df.set_index(['date','store_nbr','family']).loc[:,3,'BEVERAGES'].sales.values

In [ ]:
fig = plt.figure(figsize=(9,5))
plt.title('Beverage Sales in Store 3 by each split of the model from cross_validataion')
plt.plot(shop3_bev_cv['instance_0_prediction'], label='model 0',alpha=0.5,linestyle='dashed')
plt.plot(shop3_bev_cv['instance_1_prediction'], label='model 1',alpha=0.5,linestyle='dashed')
plt.plot(shop3_bev_cv['instance_2_prediction'], label='model 2',alpha=0.5,linestyle='dashed')
plt.plot(shop3_bev_cv['instance_3_prediction'], label='model 3',alpha=0.5,linestyle='dashed')
plt.plot(shop3_bev_cv['instance_4_prediction'], label='model 4',alpha=0.5,linestyle='dashed')

#let's plot the data from the single-train no split to compare it here as well.
plt.plot(submit_df.set_index(['date','store_nbr','family']).loc[:,3,'BEVERAGES'].sales.values, label='no_split',c='magenta')

plt.legend()
plt.show()

The later split training the prediction seems to be more inline with the better score model.

Let's see the prediction of X_train vs the real y_train

In [ ]:
y_train_s3_bev = y_.stack(['store_nbr','family']).loc[:,3,'BEVERAGES']['sales'].values

model = LinearRegression()
model.fit(X_train,y_train_s3_bev)
y_pred_s3_bev = model.predict(X_train)


fig = plt.figure(figsize=(15,8))
plt.title('BEVERAGES Sales in Store 3 from train data and from prediction')
plt.plot(y_train_s3_bev[1500:],label='y_train')
plt.plot(y_pred_s3_bev[1500:],label='y_pred',linestyle='dashed')

plt.legend()

plt.show()

The prediction of the X_train can't catch up with the peaks even in the training data.

Let's submit the last model prediction to see the score.

In [ ]:
test_data['id']

In [ ]:
pred_dic['instance_4_prediction']['id'] = test_data['id'].values

In [ ]:
#pred_dic['instance_4_prediction'][['id','sales']].to_csv('submission.csv',index=False)

got the score of 1.30819 not bad. but, still lower than the single train. However, the ref notebook got a lot better score with using only 2017 data to train. Let's try and see what gonna happen with only 2017 data.

In [ ]:
X_train_17 = X_train.loc['2017']
y_train_17_bev = y_.loc['2017'].stack(['store_nbr','family']).loc[:,3,'BEVERAGES']

model = LinearRegression()
model.fit(X_train_17,y_train_17_bev)
y_2017_pred_s3_bev = model.predict(X_train)


fig = plt.figure(figsize=(15,8))
plt.title('BEVERAGES Sales in Store 3 from 2017 train data and from prediction')
plt.plot(y_train_s3_bev[1500:],label='y_train_2017')
plt.plot(y_2017_pred_s3_bev[1500:],label='y_pred_from_2017',linestyle='dashed')

plt.legend()

plt.show()



It seems that using the late data from 2017 to train yield more accureate result to the prediction.
Let's just use the 2017 data to predict the entire targets and submit to see the change in score.

In [ ]:
model = LinearRegression()
model.fit(X_train_17,y_.loc['2017'])

y_pred_only_from_2017_data = model.predict(f_test.drop(['id','store_nbr','family','onpromotion'], axis=1))

In [ ]:
f_test.drop(['id','store_nbr','family','onpromotion'], axis=1).info()

In [ ]:
f_test

In [ ]:
X_train_17.info()

In [ ]:
X_train_17.describe()

In [ ]:
def format_prediction(array,columns_name,index,id):
    pred_ = pd.DataFrame(array)
    pred_.columns = columns_name
    pred_.index = index
    pred_ = pred_.stack(['store_nbr','family'])
    pred_.loc[pred_['sales'] < 0, 'sales'] = 0
    pred_['id'] = id
    return pred_

In [ ]:
y_pred_only_from_2017_data = format_prediction(y_pred_only_from_2017_data,y_.columns,f_test.loc['2017'].index,submit_df.set_index('date').loc['2017'].id.values)

y_pred_only_from_2017_data

Submission 

In [ ]:
#y_pred_only_from_2017_data[['id','sales']].to_csv('submission.csv',index=False)

Got the score of 1.12918 (the best so far from my code) However, something is still very wrong.

If I'm not mistaken, the ref notebook use 2017 data to train with LR and got around 0.5 RMSLE score. Too far from what I got. 

Need to find out why.


After comparing with the ref notebook there are 4 differences.

X_train/X_test - Diff

1. I didn't format the freq of the. data to 'D'
2. type_bridge of holiday which the ref didn't included
3. ref didn't drop first for the day of week data
4. week of year dtype, I use Uint32 while the ref use int64

Plan of ploblenm exploration
after some reseach on youtube, I had stumbled on this (https://www.youtube.com/watch?v=NYtwyvyvDEk) and I think i will try the not dropping first dayof week variable to see what's change in score.

if the result is still far from the ref I will try the freq format of freq = 'D'

FOr the other two (Bridge and Uint32), I still feel that they should not affect the outcome much since dtype is still numeric and Bridege Holiday is an actual holiday afterall. Therefore, I'm not planning to change that now.

In [ ]:
#current working cell

In [ ]:
y_.columns

In [ ]:
coef = pd.DataFrame(model.coef_).set_index(y_.columns)
coef.columns = X_train.columns
coef

In [ ]:
coef['type_Bridge'].describe() #bridge seem to have no weight

### Code from the ref

In [ ]:
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error

In [ ]:
df_train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', parse_dates=['date'], infer_datetime_format=True)
df_train['date'] = df_train.date.dt.to_period('D')
store_sales = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()
prom = (store_sales.groupby('date').mean().squeeze())['onpromotion'].to_frame()
average_sales = (store_sales.groupby('date').mean().squeeze())['sales'].to_frame()

In [ ]:
df_test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv', parse_dates=['date'])
df_test['date'] = df_test.date.dt.to_period('D')
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates=['date'])
oil = oil.set_index('date').to_period('d')
holiday = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv', parse_dates=['date'])
holiday = holiday.set_index('date').to_period('D')
transaction = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv', parse_dates=['date'])
transaction = transaction.set_index('date').to_period('d')

oils = oil.resample('d').mean().fillna(method='ffill')

In [ ]:
X_train = df_train.copy()
X_test = df_test.copy()
full_df = pd.concat([X_train, X_test])
full_df.reset_index(drop=True, inplace=True)
X_store = full_df.set_index(['store_nbr', 'family', 'date']).sort_index()
X_prom = (X_store.groupby('date').mean().squeeze())['onpromotion'].to_frame()
av_sales = (X_store.groupby('date').mean().squeeze())['sales'].to_frame()
full_df.head()

In [ ]:
def creat_time_features(df):
    """
    Creates time series features from datetime index
    """
    df = df.copy()
    #df['trend'] = np.arange(len(df.index))
    #df['trend^2'] = (np.arange(len(df.index)) +1) ** 2
    #df['trend^3'] = (np.arange(len(df.index)) +1) ** 3
    #df['trend^4'] = (np.arange(len(df.index)) +1) ** 4
    #df['trend^5'] = (np.arange(len(df.index)) +1) ** 5
    #df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df.index.day_of_week
    df['weekofyear'] = df.index.week
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    #df['year'] = df.index.year
    df['dayofyear'] = df.index.day_of_year
    #df['dayofmonth'] = df.index.day
    X = df.copy()
    return X
def model_cv (X, y):
    cv = TimeSeriesSplit()
    model = LinearRegression()
    result = cross_validate(model, X.loc[:'2017-07'],y.loc[:'2017-07'], cv=cv,scoring=["neg_mean_absolute_error", "neg_mean_squared_log_error"], return_estimator=True)
    mae = -result["test_neg_mean_absolute_error"]
    rmsle = np.sqrt(- result["test_neg_mean_squared_log_error"])
    print('mae = {:.3f}\nrmsle = {:.3f}'.format(mae.mean(), rmsle.mean()))
#     X_plt = X.loc['2017-08']
#     y_plt = y.loc['2017-08']
#     plt.figure(figsize=(20,15))
#     for i in result['estimator']:
#         y_pred = pd.DataFrame(i.predict(X_plt), index=X_plt.index, columns=['sales'])
#         y_pred.plot(color='red')
#         y_plt.plot(color='silver')
def X_y(df):
    df = df.copy()
    X = df.loc['2017':].dropna().drop(columns=['sales'])
    y = df.loc['2017':]['sales'].dropna()
    return X,y

In [ ]:
X_full = creat_time_features(av_sales)
X, y = X_y(X_full)
X_test = X_full.loc['2017-08-16':].drop(columns=['sales'])
model_cv(X,y);

In [ ]:
X_dum = pd.get_dummies(X, columns=['dayofweek'])
X_test = pd.get_dummies(X_test, columns=['dayofweek'])

In [ ]:

X_hol = X_dum.copy()
X_hol['new_year'] = (X_hol.index.dayofyear ==1).astype('int')
X_test['new_year'] = (X_test.index.dayofyear ==1).astype('int')

X_pay = X_hol.copy()
X_pay['day_in_m'] = X_pay.index.days_in_month
X_pay['pay_day'] = (X_pay.index.day == 16) | (X_pay.index.day == 1) | (X_pay.index.day == 14) | (X_pay.index.day == X_pay['day_in_m'] - 1) | (X_pay.index.day == 15) | (X_pay.index.day == X_pay['day_in_m'])
X_pay.drop(columns='day_in_m', inplace=True, axis=1)
X_test['day_in_m'] = X_test.index.days_in_month
X_test['pay_day'] = (X_test.index.day == 15) | (X_test.index.day == X_test['day_in_m']) | (X_test.index.day == 16) | (X_test.index.day == 1) | (X_test.index.day == 14) | (X_test.index.day == X_test['day_in_m'] - 1)
X_test.drop(columns='day_in_m', inplace=True, axis=1)

hol = holiday.loc['2017'].loc[holiday.loc['2017'].locale.isin(['National', 'Regional'])]
hol['hol'] = 1
hol.loc[hol.type == 'Transfer', 'hol'] = 1
hol.loc[(hol.type == 'Holiday') & (hol.transferred == False), 'hol'] = 1
hol.loc[(hol.type == 'Holiday') & (hol.transferred == True ), 'hol'] = 0
hol = pd.get_dummies(hol.drop(columns=['locale','locale_name','description','transferred']), columns=['type'])

X_weakend = X_pay.copy()
X_weakend = pd.concat([X_weakend, hol.loc[:'2017-08-16']], axis=1).fillna(0)
X_weakend.loc[X_weakend.index.dayofweek.isin([5,6]), 'hol'] = 1
X_test = pd.concat([X_test, hol.loc['2017-08-16':'2017-08']], axis=1).fillna(0)
X_test.loc[X_test.index.dayofweek.isin([5,6]), 'hol'] = 1

X_oil = X_weakend.copy()
X_oil['oil'] = oils.loc['2016':].rolling(7).mean()
X_test['oil']= oils.loc['2017':'2017-08'].rolling(7).mean()

In [ ]:
model = LinearRegression()
X_for_subm = X_oil.loc['2017':]
y_ref = X_store.loc[:,:,'2017':].dropna()['sales'].to_frame().unstack(['store_nbr', 'family'])

model.fit(X_for_subm, y_ref)
y_pred = pd.DataFrame(model.predict(X_for_subm), columns=y_.columns, index=X_for_subm.index)
y_pred[y_pred < 0 ] = 0
plt.figure(figsize=(20,6));
plt.plot(y_.loc(axis=1)['sales',1, 'BEVERAGES'].loc['2017'].values)
plt.plot(y_pred.loc(axis=1)['sales',1, 'BEVERAGES'].loc['2017'].values)
mae = mean_absolute_error(y_pred.loc(axis=1)['sales',1, 'PRODUCE'].loc['2017'].values,y_.loc(axis=1)['sales',1, 'PRODUCE'].loc['2017'].values)
msle = mean_squared_log_error(y_pred.loc(axis=1)['sales',1, 'PRODUCE'].loc['2017'].values,y_.loc(axis=1)['sales',1, 'PRODUCE'].loc['2017'].values)
print('mae = {:.3f}\nrmsle = {:.3f}'.format(mae, np.sqrt(msle)))

Compare the df

In [ ]:
X_train_compare_with_ref = X_train_17.drop('type_Bridge',axis=1)

In [ ]:
X_train_compare_with_ref.columns  = X_for_subm.columns

In [ ]:
#X_train_compare_with_ref.index = X_train_compare_with_ref.index.to_period('D')

In [ ]:
X_train_compare_with_ref['dayofyear'] = X_train_compare_with_ref['dayofyear'].astype('int64')

In [ ]:
X_train_compare_with_ref.info()

In [ ]:
X_for_subm.info()

let's try training the model with the new exact same inputs.

In [ ]:
model = LinearRegression()
model.fit(X_train_compare_with_ref,y_.loc['2017'])

y_pred_from_x_train_compare_with_ref = model.predict(f_test.drop(['id','store_nbr','family','onpromotion','type_Bridge'], axis=1))

In [ ]:
y_pred_from_x_train_compare_with_ref

In [ ]:
#submit_from_x_train_compare_with_ref = format_prediction(y_pred_from_x_train_compare_with_ref,y_.columns,f_test.loc['2017'].index,submit_df.set_index('date').loc['2017'].id.values)

In [ ]:
#submit_from_x_train_compare_with_ref[['id','sales']].to_csv('submission_mirror.csv',index=False)

The score is still a bit bad even thought the inputs are mirrored. Let's check the y_ train target.

In [ ]:
y_.loc['2017'].shape

In [ ]:
y_ref.shape

the shaps is the same let's check the content

In [ ]:
y_2017 = y_.loc['2017']
y_2017.tail()

In [ ]:
y_ref.tail()

I notice the difference in columns

In [ ]:
y_2017.columns

In [ ]:
y_ref.columns

Seems like I didn't sort the column. so the last shop in my target is shop#9 while it's supposed to be shop #59

Let's do it

In [ ]:
y_2017_sort = y_2017.copy()
y_2017_sort = y_2017_sort.sort_index(axis=1,level=1)

y_2017_sort

In [ ]:
model = LinearRegression()
model.fit(X_train_compare_with_ref,y_2017_sort)

In [ ]:
pred_from_y_2017_sort = model.predict(f_test.drop(['id','store_nbr','family','onpromotion','type_Bridge'], axis=1))

In [ ]:
submit_from_y_2017_sort = format_prediction(pred_from_y_2017_sort,y_2017_sort.columns,f_test.index,submit_df.set_index('date').loc['2017'].id.values)

In [ ]:
submit_from_y_2017_sort[['id','sales']].to_csv('submission_mirror.csv',index=False)

~~Still, the score didn't improve. It's stays the same. let's compare everything with the ref again.~~

Training data are the same. Chcek the training target.

In [ ]:
#y_ref.values == y_2017_sort.values

A lot of false here.